In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
file_path = './data/'

In [3]:
currency_pair = 'Aud_Usd'
years = '2021-2022'

In [4]:
def atr(high, low, close, lookback=14):
    high_low = high - low
    high_close = np.abs(high - close.shift())
    low_close = np.abs(low - close.shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)

    return true_range.rolling(lookback).sum() / lookback

def adx(high, low, close, lookback=14):
    plus_dm = high.diff()
    minus_dm = low.diff()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm > 0] = 0
    
    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift(1)))
    tr3 = pd.DataFrame(abs(low - close.shift(1)))
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis = 1, join = 'inner').max(axis = 1)
    atr = tr.rolling(lookback).mean()
    
    plus_di = 100 * (plus_dm.ewm(alpha = 1/lookback).mean() / atr)
    minus_di = abs(100 * (minus_dm.ewm(alpha = 1/lookback).mean() / atr))
    dx = (abs(plus_di - minus_di) / abs(plus_di + minus_di)) * 100
    adx = ((dx.shift(1) * (lookback - 1)) + dx) / lookback
    adx_smooth = adx.ewm(alpha = 1/lookback).mean()

    return adx_smooth

def chop(df, lookback=14):
    atr1 = atr(df['Mid_High'], df['Mid_Low'], df['Mid_Close'], lookback=1)
    high, low = df['Mid_High'], df['Mid_Low']

    chop = np.log10(atr1.rolling(lookback).sum() / (high.rolling(lookback).max() - low.rolling(lookback).min())) / np.log10(lookback)

    return chop

def vo(volume, short_lookback=5, long_lookback=10):
    short_ema =  pd.Series.ewm(volume, span=short_lookback).mean()
    long_ema = pd.Series.ewm(volume, span=long_lookback).mean()

    volume_oscillator = (short_ema - long_ema) / long_ema

    return volume_oscillator

In [5]:
def apply_impact(val, impact):
    return float(val) * float(impact)

In [6]:
currency1, currency2 = currency_pair.split('_')
currency1, currency2 = currency1.upper(), currency2.upper()

news = pd.read_csv(file_path + 'events.csv')
news.Date = pd.to_datetime(news.Date, utc=True)
news.drop(news[(news['Impact'] != 'low') & (news['Impact'] != 'med') & (news['Impact'] != 'high')].index, inplace=True)
news.loc[news['Impact'] == 'low', 'Impact'] = 1
news.loc[news['Impact'] == 'med', 'Impact'] = 2
news.loc[news['Impact'] == 'high', 'Impact'] = 3
news['Impact'] = pd.to_numeric(news['Impact'])
news['Actual_Class'] = news.apply(lambda row: apply_impact(row['Actual'], row['Impact']), axis=1)
news['Previous_Class'] = news.apply(lambda row: apply_impact(row['Previous'], row['Impact']), axis=1)
news_base = news.loc[news['Currency_Code'] == currency1]
news_counter = news.loc[news['Currency_Code'] == currency2]
news_base.drop(['Currency_Code', 'Actual', 'Previous', 'Actual_Val', 'Forecast_Val', 'Previous_Val'], axis=1, inplace=True)
news_counter.drop(['Currency_Code', 'Actual', 'Previous', 'Actual_Val', 'Forecast_Val', 'Previous_Val'], axis=1, inplace=True)
by_date1 = news_base.groupby('Date')
impact1, actual1, previous1 = by_date1['Impact'].mean().reset_index(), by_date1['Actual_Class'].mean().reset_index(), by_date1['Previous_Class'].mean().reset_index()
news_base = news_base.iloc[0:0]
news_base['Date'], news_base['Impact'], news_base['Actual_Class'], news_base['Previous_Class'] = impact1['Date'], impact1['Impact'], actual1['Actual_Class'], previous1['Previous_Class']
by_date2 = news_counter.groupby('Date')
impact2, actual2, previous2 = by_date2['Impact'].mean().reset_index(), by_date2['Actual_Class'].mean().reset_index(), by_date2['Previous_Class'].mean().reset_index()
news_counter = news_counter.iloc[0:0]
news_counter['Date'], news_counter['Impact'], news_counter['Actual_Class'], news_counter['Previous_Class'] = impact2['Date'], impact2['Impact'], actual2['Actual_Class'], previous2['Previous_Class']

/Users/mymac/miniforge3/envs/forex/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-6-42e2058cb015>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_base['Date'], news_base['Impact'], news_base['Actual_Class'], news_base['Previous_Class'] = impact1['Date'], impact1['Impact'], actual1['Actual_Class'], previous1['Previous_Class']
<ipython-input-6-42e2058cb015>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [7]:
df = pd.read_csv(file_path + f'Oanda_{currency_pair}_M5_{years}.csv')
df.Date = pd.to_datetime(df.Date, utc=True)
df.reset_index(drop=True, inplace=True)

df['atr'] = atr(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
df['adx'] = adx(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
df['chop'] = chop(df)
df['vo'] = vo(df['Volume'])

df = pd.merge(df, news_base, how='left', on='Date')
df = pd.merge(df, news_counter, how='left', on='Date')
df.reset_index(drop=True, inplace=True)
df = df.fillna(method='ffill')
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

df_long = pd.read_csv(file_path + f'Oanda_{currency_pair}_H1_{years}.csv')
df_long.Date = pd.to_datetime(df_long.Date, utc=True)
df_long.reset_index(drop=True, inplace=True)

df_long['atr'] = atr(df_long['Mid_High'], df_long['Mid_Low'], df_long['Mid_Close'])
df_long['adx'] = adx(df_long['Mid_High'], df_long['Mid_Low'], df_long['Mid_Close'])
df_long['chop'] = chop(df_long)
df_long['vo'] = vo(df_long['Volume'])

df_long.dropna(inplace=True)
df_long.reset_index(drop=True, inplace=True)

In [8]:
df

,Date,Bid_Open,Bid_High,Bid_Low,Bid_Close,Ask_Open,Ask_High,Ask_Low,Ask_Close,Mid_Open,...,atr,adx,chop,vo,Impact_x,Actual_Class_x,Previous_Class_x,Impact_y,Actual_Class_y,Previous_Class_y
0,2021-04-02 00:00:00+00:00,0.76162,0.76187,0.76161,0.76181,0.76176,0.76201,0.76176,0.76194,0.76169,...,0.000134,14.006212,0.556982,0.168798,1.0,0.0,0.0,2.0,2.0,0.0
1,2021-04-02 00:05:00+00:00,0.76184,0.76288,0.76184,0.76278,0.76198,0.76304,0.76198,0.76293,0.76191,...,0.000199,14.776484,0.247997,0.527475,1.0,0.0,0.0,2.0,2.0,0.0
2,2021-04-02 00:10:00+00:00,0.76275,0.76302,0.76272,0.76299,0.76289,0.76316,0.76287,0.76313,0.76282,...,0.000210,18.154826,0.235306,0.451073,1.0,0.0,0.0,2.0,2.0,0.0
3,2021-04-02 00:15:00+00:00,0.76302,0.76316,0.76292,0.76293,0.76316,0.76331,0.76307,0.76307,0.76309,...,0.000217,21.471467,0.213613,0.288505,1.0,0.0,0.0,2.0,2.0,0.0
4,2021-04-02 00:20:00+00:00,0.76295,0.76326,0.76295,0.76326,0.76312,0.76342,0.76312,0.76342,0.76304,...,0.000233,24.737933,0.218794,0.127286,1.0,0.0,0.0,2.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74301,2022-04-01 05:35:00+00:00,0.74750,0.74773,0.74747,0.74767,0.74765,0.74787,0.74761,0.74781,0.74758,...,0.000291,29.258498,0.457454,0.014141,1.0,0.0,0.0,1.0,0.0,0.0
74302,2022-04-01 05:40:00+00:00,0.74768,0.74768,0.74740,0.74756,0.74780,0.74783,0.74755,0.74769,0.74774,...,0.000298,29.944695,0.484831,0.071273,1.0,0.0,0.0,1.0,0.0,0.0
74303,2022-04-01 05:45:00+00:00,0.74756,0.74770,0.74754,0.74766,0.74770,0.74787,0.74768,0.74781,0.74763,...,0.000295,30.721020,0.545073,0.048300,1.0,0.0,0.0,1.0,0.0,0.0
74304,2022-04-01 05:50:00+00:00,0.74764,0.74766,0.74734,0.74762,0.74780,0.74784,0.74748,0.74777,0.74772,...,0.000277,31.295025,0.607905,0.031598,1.0,0.0,0.0,1.0,0.0,0.0


In [9]:
df_long

,Date,Bid_Open,Bid_High,Bid_Low,Bid_Close,Ask_Open,Ask_High,Ask_Low,Ask_Close,Mid_Open,Mid_High,Mid_Low,Mid_Close,Volume,atr,adx,chop,vo
0,2021-01-04 12:00:00+00:00,0.77226,0.77236,0.77135,0.77152,0.77240,0.77251,0.77147,0.77163,0.77233,0.77244,0.77141,0.77158,515,0.001673,4.219022,0.519838,-0.091131
1,2021-01-04 13:00:00+00:00,0.77148,0.77258,0.77126,0.77174,0.77160,0.77274,0.77140,0.77188,0.77154,0.77266,0.77134,0.77181,673,0.001605,6.965572,0.504147,-0.043919
2,2021-01-04 14:00:00+00:00,0.77177,0.77220,0.77031,0.77046,0.77192,0.77234,0.77047,0.77061,0.77184,0.77226,0.77039,0.77054,1307,0.001504,7.261728,0.518343,0.109708
3,2021-01-04 15:00:00+00:00,0.77048,0.77068,0.76797,0.76838,0.77067,0.77084,0.76811,0.76853,0.77058,0.77076,0.76804,0.76846,1311,0.001481,11.252024,0.467462,0.160819
4,2021-01-04 16:00:00+00:00,0.76833,0.76834,0.76532,0.76541,0.76850,0.76850,0.76555,0.76560,0.76842,0.76842,0.76544,0.76550,1317,0.001619,18.561803,0.365357,0.170148
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6217,2021-12-31 17:00:00+00:00,0.72687,0.72768,0.72659,0.72768,0.72701,0.72781,0.72674,0.72781,0.72694,0.72774,0.72666,0.72774,208,0.001078,16.362057,0.577155,0.114378
6218,2021-12-31 18:00:00+00:00,0.72763,0.72763,0.72710,0.72744,0.72777,0.72777,0.72725,0.72758,0.72770,0.72770,0.72718,0.72751,137,0.001089,16.790004,0.580903,-0.029509
6219,2021-12-31 19:00:00+00:00,0.72748,0.72752,0.72685,0.72700,0.72762,0.72766,0.72702,0.72716,0.72755,0.72758,0.72694,0.72708,109,0.001089,17.160473,0.580903,-0.144993
6220,2021-12-31 20:00:00+00:00,0.72704,0.72730,0.72654,0.72654,0.72718,0.72746,0.72669,0.72669,0.72711,0.72738,0.72662,0.72662,117,0.001116,17.119836,0.590235,-0.218318


In [10]:
rounding = 5 if 'Jpy' not in currency_pair else 3
divider = 10000 if 'Jpy' not in currency_pair else 100
value_per_pip = 1.0
amounts_per_day = [-0.00008, -0.0001, -0.00012]

In [12]:
# ----------------------------------------------------------------------------------------------------
# Simulation code
# ----------------------------------------------------------------------------------------------------
def calculate_day_fees(start_date, end_date, n_units):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25]) * n_units
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee

def get_n_units(trade_type, stop_loss, ask_open, bid_open, mid_open, currency_pair):
    _, second = currency_pair.split('_')
  
    pips_to_risk = ask_open - stop_loss if trade_type == 'buy' else stop_loss - bid_open
    pips_to_risk_calc = pips_to_risk * 10000 if second != 'Jpy' else pips_to_risk * 100

    if second == 'Usd':
        per_pip = 0.0001

    else:
        per_pip = 0.0001 / mid_open if second != 'Jpy' else 0.01 / mid_open

    n_units = int(50 / (pips_to_risk_calc * per_pip))

    return n_units

def run_simulation(risk_reward_ratio, spread_cutoff, n_bars, pip_movement, use_pullback, pips_to_risk, pullback_percentage, volume_cutoff, chop_cutoff, adx_cutoff, use_news, use_long):
    reward = 0
    n_wins = 0
    n_losses = 0
    win_streak = 0
    loss_streak = 0
    curr_win_streak = 0
    curr_loss_streak = 0
    n_buys = 0
    n_sells = 0
    pips_risked = []
    day_fees = 0
    trade = None
    lookback = n_bars + 1 if use_pullback else n_bars
    lookforward = -1 if use_pullback else 0
    pip_movement /= divider
    pips_to_risk = pips_to_risk / divider if type(pips_to_risk) != str else pips_to_risk

    for i in range(lookback, len(df)):
        curr_bid_open, curr_bid_high, curr_bid_low, curr_ask_open, curr_ask_high, curr_ask_low, curr_mid_open, curr_date = df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Ask_Open', 'Ask_High', 'Ask_Low', 'Mid_Open', 'Date']]

        if use_long:
            if len(df_long.loc[df_long.Date >= curr_date]) == 0:
                break

            curr_long = df_long.loc[df_long.Date <= curr_date]
            curr_long.reset_index(drop=True, inplace=True)

            if len(curr_long) < 2:
                continue

            actual_base, actual_counter = df.loc[df.index[i - 1], ['Actual_Class_x', 'Actual_Class_y']]
            volume, chop, adx = curr_long.loc[curr_long.index[-2], ['vo', 'chop', 'adx']]

        else:
            volume, chop, adx, actual_base, actual_counter = df.loc[df.index[i - 1], ['vo', 'chop', 'adx', 'Actual_Class_x', 'Actual_Class_y']]
        
        spread = abs(curr_ask_open - curr_bid_open)

        mid_opens = list(df.loc[df.index[i - lookback:i + lookforward], 'Mid_Open'])
        mid_highs = list(df.loc[df.index[i - lookback:i + lookforward], 'Mid_High'])
        mid_lows = list(df.loc[df.index[i - lookback:i + lookforward], 'Mid_Low'])
        mid_closes = list(df.loc[df.index[i - lookback:i + lookforward], 'Mid_Close'])

        buy_signal = all([mid_opens[j] < mid_closes[j] for j in range(len(mid_opens))]) and abs(mid_opens[0] - mid_closes[-1]) >= pip_movement
        sell_signal = all([mid_opens[j] > mid_closes[j] for j in range(len(mid_opens))]) and abs(mid_opens[0] - mid_closes[-1]) >= pip_movement

        if use_pullback and buy_signal:
            mid_open1, mid_high1, mid_low1, mid_close1 = df.loc[df.index[i - 1], ['Mid_Open', 'Mid_High', 'Mid_Low', 'Mid_Close']]
            buy_signal = mid_open1 > mid_close1 and abs(mid_close1 - mid_open1) <= pullback_percentage * abs(mid_high1 - mid_low1)

        if use_pullback and sell_signal:
            mid_open1, mid_high1, mid_low1, mid_close1 = df.loc[df.index[i - 1], ['Mid_Open', 'Mid_High', 'Mid_Low', 'Mid_Close']]
            sell_signal = mid_open1 < mid_close1 and abs(mid_close1 - mid_open1) <= pullback_percentage * abs(mid_high1 - mid_low1)

        volume_signal = volume >= volume_cutoff if volume_cutoff is not None else True
        chop_signal = chop <= chop_cutoff if chop_cutoff is not None else True
        adx_signal = adx >= adx_cutoff if adx_cutoff is not None else True
        news_buy = actual_base >= 0 if use_news else True
        news_sell = actual_base <= 0 if use_news else True

        buy_signal = buy_signal and volume_signal and chop_signal and adx_signal and news_buy
        sell_signal = sell_signal and volume_signal and chop_signal and adx_signal and news_sell

        highest_high, lowest_low = max(mid_highs), min(mid_lows)

        if trade is None:
            if buy_signal:
                open_price = float(curr_ask_open)

                stop_loss = open_price - pips_to_risk if type(pips_to_risk) != str else lowest_low
                stop_loss = round(stop_loss, rounding)

                if stop_loss < open_price:
                    curr_pips_to_risk = open_price - stop_loss

                    if spread <= curr_pips_to_risk * spread_cutoff:
                        stop_gain = round(open_price + (risk_reward_ratio * curr_pips_to_risk), rounding)
                        n_units = get_n_units('buy', stop_loss, curr_ask_open, curr_bid_open, curr_mid_open, currency_pair)

                        trade = {'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                                        'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, rounding),
                                                        'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

                        n_buys += 1

                        pips_risked.append(curr_pips_to_risk)

            elif sell_signal:
                open_price = float(curr_bid_open)
                
                stop_loss = open_price + pips_to_risk if type(pips_to_risk) != str else highest_high
                stop_loss = round(stop_loss, rounding)

                if stop_loss > open_price:
                    curr_pips_to_risk = stop_loss - open_price

                    if spread <= curr_pips_to_risk * spread_cutoff:
                        stop_gain = round(open_price - (risk_reward_ratio * curr_pips_to_risk), rounding)
                        n_units = get_n_units('sell', stop_loss, curr_ask_open, curr_bid_open, curr_mid_open, currency_pair)

                        trade = {'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, rounding),
                                'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

                        n_sells += 1

                        pips_risked.append(curr_pips_to_risk)

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
            trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
            # trade_amount = -50.0
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None    
            closed_trade = True

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
            trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
            # trade_amount = 50.0 * risk_reward_ratio
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
            trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
            # trade_amount = -50.0
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
            trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
            # trade_amount = 50.0 * risk_reward_ratio
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

    return reward + day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked

In [13]:
# ----------------------------------------------------------------------------------------------------
# Run simulation
# ----------------------------------------------------------------------------------------------------
risk_reward_ratio_vals = [1.5]
spread_cutoffs = [0.10]
n_bars_vals = [3]
pip_movement_vals = [20]
use_pullback_vals = [True]
pips_to_risk_vals = ['bars']  # LEAVE THE 'bars' VALUE IN, YOU CAN CHANGE THE OTHER VALUES
pullback_percentages = [0.50]
# NEW STUFF
volume_cutoffs = [None]  # Cutoff for VO indicator - values should be between 0 and 1, higher is more restrictive
chop_cutoffs = [0.50]  # Cutoff for chop indicator - values should be between 0 and 1, lower is more restrictive
adx_cutoffs = [20]  # Cutoff for adx indicator - values should be between 0 and 100, higher is more restrictive
use_news_vals = [True]  # Whether or not to check the news when making a trade
use_long_vals = [False]  # Whether or not to use a longer timeframe when looking at VO, chop, and adx

n_possibilities = len(risk_reward_ratio_vals) * len(spread_cutoffs) * len(n_bars_vals) * len(pip_movement_vals) * len(use_pullback_vals) * len(pips_to_risk_vals) * len(pullback_percentages) * len(volume_cutoffs) * len(chop_cutoffs) * len(adx_cutoffs) * len(use_news_vals) * len(use_long_vals)
all_combos = []

for risk_reward_ratio in risk_reward_ratio_vals:
    for spread_val in spread_cutoffs:
        for n_bars in n_bars_vals:
            for pip_movement in pip_movement_vals:
                for use_pullback in use_pullback_vals:
                    for pips_to_risk in pips_to_risk_vals:
                        for pullback_percentage in pullback_percentages:
                            for volume_cutoff in volume_cutoffs:
                                for chop_cutoff in chop_cutoffs:
                                    for adx_cutoff in adx_cutoffs:
                                        for use_news in use_news_vals:
                                            for use_long in use_long_vals:
                                                all_combos.append((risk_reward_ratio, spread_val, n_bars, pip_movement, use_pullback, pips_to_risk, pullback_percentage, volume_cutoff, chop_cutoff, adx_cutoff, use_news, use_long))

percentage_to_try = 1
n_runs = int(percentage_to_try * len(all_combos))
combos_to_try = random.sample(all_combos, n_runs)
print('Num runs: '+ str(len(combos_to_try)) + '\n')

best_risk_reward = None
best_spread_cutoff = None
best_n_bars_val = None
best_pip_movement_val = None
best_use_pullback_val = None
best_pips_to_risk = None
best_pullback_percentage = None
best_volume_cutoff = None
best_chop_cutoff = None
best_adx_cutoff = None
best_use_news_val = None
best_use_long_val = None
top_n_results = 10
best_rewards = []
best_reward = -np.inf
runs_finished = 0

for risk_reward_ratio, spread_val, n_bars, pip_movement, use_pullback, pips_to_risk, pullback_percentage, volume_cutoff, chop_cutoff, adx_cutoff, use_news, use_long in combos_to_try:
    reward, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked = run_simulation(risk_reward_ratio, spread_val, n_bars, pip_movement, use_pullback, pips_to_risk, pullback_percentage, volume_cutoff, chop_cutoff, adx_cutoff, use_news, use_long)
    runs_finished += 1

    print(reward)
    print('Num buys: ' + str(n_sells))
    print('Num sells: ' + str(n_buys))
    print('Num trades: ' + str(n_buys + n_sells))
    print('Num wins: ' + str(n_wins))
    print('Num losses: ' + str(n_losses))
    print('Win streak: ' + str(win_streak))
    print('Loss streak: ' + str(loss_streak))
    if len(pips_risked) > 0:
        print('Avg pips risked: ' + str(np.array(pips_risked).mean()))
    print('Remaining runs: ' + str(n_runs - runs_finished))

    min_item = min(best_rewards, key=lambda entry: entry['reward']) if len(best_rewards) >= top_n_results else None

    if min_item is None or reward > min_item['reward']:
        if min_item is not None:
            best_rewards.remove(min_item)
            
        best_rewards.append({'reward': int(reward), 'ratio': risk_reward_ratio, 'spread': spread_val, 'n_bars': n_bars, 'pip_movement': pip_movement, 'use_pullback': use_pullback, 'pips_to_risk': pips_to_risk, 'pullback_percentage': pullback_percentage, 'volume_cutoff': volume_cutoff, 'chop_cutoff': chop_cutoff, 'adx_cutoff': adx_cutoff, 'use_news': use_news, 'use_long': use_long})


    if reward > best_reward:
        best_reward = reward
        best_risk_reward = risk_reward_ratio
        best_spread_cutoff = spread_val
        best_n_bars_val = n_bars
        best_pip_movement_val = pip_movement
        best_use_pullback_val = use_pullback
        best_pips_to_risk = pips_to_risk
        best_pullback_percentage = pullback_percentage
        best_volume_cutoff = volume_cutoff
        best_chop_cutoff = chop_cutoff
        best_adx_cutoff = adx_cutoff
        best_use_news_val = use_news
        best_use_long_val = use_long

    print('Best reward so far: ' + str(best_reward))
    print()

Num runs: 1

11.227259999993628
Num buys: 6
Num sells: 6
Num trades: 12
Num wins: 5
Num losses: 7
Win streak: 4
Loss streak: 7
Avg pips risked: 0.002624166666666696
Remaining runs: 0
Best reward so far: 11.227259999993628



In [14]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best n bars val: ' + str(best_n_bars_val))
print('Best pip movement val: ' + str(best_pip_movement_val))
print('Best use pullback val: ' + str(best_use_pullback_val))
print('Best pips to risk: ' + str(best_pips_to_risk))
print('Best pullback percentage: ' + str(best_pullback_percentage))
print('Best volume cutoff: ' + str(best_volume_cutoff))
print('Best chop cutoff: ' + str(best_chop_cutoff))
print('Best adx cutoff: ' + str(best_adx_cutoff))
print('Best use news val: ' + str(best_use_news_val))
print('Best use long val: ' + str(best_use_long_val))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 11.227259999993628
Best risk/reward ratio: 1.5
Best spread: 0.1
Best n bars val: 3
Best pip movement val: 20
Best use pullback val: True
Best pips to risk: bars
Best pullback percentage: 0.5
Best volume cutoff: None
Best chop cutoff: 0.5
Best adx cutoff: 20
Best use news val: True
Best use long val: False
-----------------------
Top results:
{'reward': 11, 'ratio': 1.5, 'spread': 0.1, 'n_bars': 3, 'pip_movement': 20, 'use_pullback': True, 'pips_to_risk': 'bars', 'pullback_percentage': 0.5, 'volume_cutoff': None, 'chop_cutoff': 0.5, 'adx_cutoff': 20, 'use_news': True, 'use_long': False}
